In [133]:
import pandas as pd
import recordlinkage as rl
import json

### Read two geonames files and concat them to one data frame

In [134]:
header = ['geonameid','name','asciiname','alternatenames','latitude','longitude','feature class','feature code','country code','cc2','admin1 code','admin2 code','admin3 code','admin4 code','population','elevation','dem','timezone','modification date']
geonamesFirst = pd.read_csv('D:/Geonames/modifications-2018-02-24.txt', delimiter='\t', header=None, names=header, low_memory=False)
geonamesSecond = pd.read_csv('D:/Geonames/modifications-2018-02-27.txt', delimiter='\t', header=None, names=header, low_memory=False)
geonamesUnion = pd.concat([geonamesFirst, geonamesSecond])
#len(geonamesUnion) - len(geonamesFirst) - len(geonamesSecond)
len(geonamesUnion)

2380

### Ensure an unique index

In [135]:
geonamesFirst.reset_index(drop=True, inplace=True)
geonamesSecond.reset_index(drop=True, inplace=True)
geonamesUnion.reset_index(drop=True, inplace=True)
len(geonamesUnion)

2380

### Create the cartesian product indexer

In [136]:
indexer = rl.Index()
#indexer.block('geonameid')
indexer.full()
blocks = indexer.index(geonamesFirst, geonamesUnion)
len(blocks)

3803240

### Find matching candidates

In [137]:
comparer = rl.Compare()
comparer.exact('geonameid', 'geonameid', label='id')
comparer.string('name', 'name', method='jarowinkler', threshold=0.85, label='name')
comparer.string('asciiname', 'asciiname', method='jarowinkler', threshold=0.85, label='ascii')
comparer.exact('latitude', 'latitude', label='lat')
comparer.exact('longitude', 'longitude', label='lon')
minMatchScore = 3.0
compareResult = comparer.compute(blocks, geonamesFirst, geonamesUnion)
print(compareResult.describe())
compareResult.sum(axis=1).value_counts().sort_index(ascending=False)
matches = compareResult[compareResult.sum(axis=1) >= minMatchScore]
print(matches.head())
print(len(matches))

                 id          name         ascii           lat           lon
count  3.803240e+06  3.803240e+06  3.803240e+06  3.803240e+06  3.803240e+06
mean   4.201681e-04  1.412454e-02  1.608891e-02  2.053302e-02  2.000531e-02
std    2.049370e-02  1.180044e-01  1.258176e-01  1.418147e-01  1.400182e-01
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
50%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
75%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
max    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00
     id  name  ascii  lat  lon
0 0   1   1.0    1.0    1    1
1 1   1   1.0    1.0    1    1
2 2   1   1.0    1.0    1    1
3 3   1   1.0    1.0    1    1
4 4   1   1.0    1.0    1    1
5282


### Obtain records by using the multi-index of the matches and put those pairs into a dictionary

In [161]:
def toDict(series):
    seriesAsDict = series.to_dict()
    for key, value in seriesAsDict.items():
        seriesAsDict[key] = str(value)
    return seriesAsDict

matchPairs = {}
for firstId, secondId in matches.index.get_values():
    leftRecord = geonamesFirst.loc[firstId]
    rightRecord = geonamesUnion.loc[secondId]
    if (firstId in matchPairs):
        pairs = matchPairs[firstId]
        pairs.append(toDict(rightRecord))
    else:
        pairs = [toDict(leftRecord), toDict(rightRecord)]
        matchPairs[firstId] = pairs

len(matchPairs)

1598

### Write the pairs to a json file

In [162]:
indentLevel = 2
with open('D:/Geonames/recordlinkage-results.json', 'w', encoding='utf8') as jsonFile:
    json.dump(matchPairs, jsonFile, ensure_ascii=False, indent=indentLevel)